In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as f
from torch import Tensor

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import pickle

In [2]:
#1.测试user是否可以编码
##1.1读取所有用户数据
dir_user='/home/jovyan/datasets/tsmc_nyc_3_groupby_user_chronological/user_id.pkl'
with open(dir_user,'rb') as f:
    users=pickle.load(f)
users=torch.IntTensor(users)

In [3]:
print(users)
print(len(users))

tensor([   1,    2,    3,  ..., 1081, 1082, 1083], dtype=torch.int32)
1078


In [4]:
##1.2用户数据量
# print(len(users))#1078
# size_user=len(users)
size_user=max(users)+1
##1.3测试编码
embed_size_user=50
user_embedder=nn.Embedding(size_user,embed_size_user)
user_embedding=user_embedder(users)

In [5]:
#2测试cat是否可以编码，cat包含十六进制
##2.1读字典数据
dir_cat='/home/jovyan/datasets/tsmc_nyc_6_distinguish_cat_poi/distin_dic_cat_id.pkl'
with open(dir_cat,'rb') as f:
    dic_cat_id=pickle.load(f)
print(dic_cat_id)

{'4bf58dd8d48988d157941735': 0, '4bf58dd8d48988d1f1931735': 1, '4bf58dd8d48988d14e941735': 2, '4bf58dd8d48988d143941735': 3, '4bf58dd8d48988d1cc941735': 4, '4bf58dd8d48988d1e0931735': 5, '4bf58dd8d48988d11d941735': 6, '4bf58dd8d48988d113941735': 7, '4bf58dd8d48988d11b941735': 8, '4bf58dd8d48988d1fa931735': 9, '4bf58dd8d48988d116941735': 10, '4bf58dd8d48988d16c941735': 11, '4bf58dd8d48988d163941735': 12, '4bf58dd8d48988d147941735': 13, '4bf58dd8d48988d10c941735': 14, '4bf58dd8d48988d14c941735': 15, '4bf58dd8d48988d1c1941735': 16, '4bf58dd8d48988d123941735': 17, '4bf58dd8d48988d1ce941735': 18, '4bf58dd8d48988d155941735': 19, '4bf58dd8d48988d11e941735': 20, '5032792091d4c4b30a586d5c': 21, '4bf58dd8d48988d1f7941735': 22, '4bf58dd8d48988d117941735': 23, '4bf58dd8d48988d16f941735': 24, '4bf58dd8d48988d180941735': 25, '4bf58dd8d48988d1d4941735': 26, '4bf58dd8d48988d1ca941735': 27, '4bf58dd8d48988d190941735': 28, '4bf58dd8d48988d1ed931735': 29, '4bf58dd8d48988d16a941735': 30, '4bf58dd8d48988d1

In [6]:
##查看数据
print(type(dic_cat_id))

<class 'dict'>


In [7]:
##2.2生成index列表和IntTensor
cat_id=[i for i in range(len(dic_cat_id))]
cat_id=torch.IntTensor(cat_id)

In [8]:
print(type(cat_id))

<class 'torch.Tensor'>


In [9]:
##2.3cat嵌入
size_cat=len(cat_id)
embed_size_cat=100
cat_embeder=nn.Embedding(size_cat,embed_size_cat)
cat_embedding=cat_embeder(cat_id)

In [10]:
#3测试能否拼接
##3.1读delta_t数据
dir_rec_delta_t='/home/jovyan/datasets/tsmc_nyc_5_deltat_delta_d/rec_delta_t.pkl'
with open(dir_rec_delta_t,'rb') as f:
    rec_delta_t=pickle.load(f)

In [11]:
##3.2做padding函数，返回padded数据和valid_len
def Padding_valid_len(data):
    '''
    在data最后一维对齐
    '''
    max_len,lens=Find_maxlen_lens(data)
    padded_data=Padding(data,max_len,lens)
    # padded_data=Tensor(padded_data)
    return padded_data,max_len,lens

In [12]:
#3.2.1计算max_len和valid_len
def Find_maxlen_lens(data,lens=None):
    if lens is None:
        lens=[]
    if not isinstance(data,list):
        return 0,lens
    if not data:
        return 0,lens
    if all(not isinstance(sublist,list) for sublist in data):
        length=len(data)
        lens.append(length)
        return length,lens
    max_len=max(Find_maxlen_lens(sublist,lens)[0] for sublist in data)
    return max_len,lens

In [13]:
##3.2.3根据maxlen，valid_len,用0做padding
def Padding(data,max_len,lens):
    if not isinstance(data,list):
        return data
    if all(not isinstance(sublist,list) for sublist in data):
        return data+[0]*(max_len-len(data))
    return torch.IntTensor([Padding(sublist,max_len,lens) for sublist in data])

In [14]:
##3.2.2测试返回结果
nested_list = [[1, 2, 3], [4, 5, 6, 7], [10, 11, 12, 13, 14], [15, 16, 17, 18]]
padded,max_length,lens = Padding_valid_len(nested_list)
# padded=Tensor(padded)
print("最小维度上的最大长度:", max_length)
print(lens)
print(f'padded:{padded}')

最小维度上的最大长度: 5
[3, 4, 5, 4]
padded:tensor([[ 1,  2,  3,  0,  0],
        [ 4,  5,  6,  7,  0],
        [10, 11, 12, 13, 14],
        [15, 16, 17, 18,  0]], dtype=torch.int32)


In [15]:
#3.3做delta_t的padding和valid_len
padded_rec_delta_t,max_len,lens=Padding_valid_len(rec_delta_t)#max_len,lens都一样
# print(f"max_len:{max_len},len:{lens}")
print(f'padded_delta_t:{padded_rec_delta_t}')

padded_delta_t:tensor([[   67613,    87274,   356947,  ...,   435024, 10751110,   162465],
        [ 7512611,  3876121,    24153,  ...,   685830,    13488,     3423],
        [    4092,   344985,   253180,  ...,    75781,   476178,  3580557],
        ...,
        [  444307,   665732,   333925,  ...,   485888,        0,        0],
        [     465,       47,   129815,  ...,      140,   206693,   929301],
        [  102170,    17419,  1082673,  ...,     1742,   499759,    49031]],
       dtype=torch.int32)


/tmp/ipykernel_377/2476922.py:7: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return torch.IntTensor([Padding(sublist,max_len,lens) for sublist in data])


In [16]:
#做delta_d的padding
#max_len,lens一样，用padding就可以
#读delta_d数据
dir_rec_delta_d='/home/jovyan/datasets/tsmc_nyc_5_deltat_delta_d/rec_delta_d.pkl'
with open(dir_rec_delta_d,'rb') as f:
    rec_delta_d=pickle.load(f)
padded_rec_delta_d=Padding(rec_delta_d,max_len,lens)

/tmp/ipykernel_377/2476922.py:7: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return torch.IntTensor([Padding(sublist,max_len,lens) for sublist in data])


In [17]:
#4做rec_cat_id的padding
#4.1读rec_cat_id_reindex(重编码的cat_id)
dir_rec_cat_id_reindex='/home/jovyan/datasets/tsmc_nyc_4_recent_target/rec_cat_id_reindex.pkl'
with open(dir_rec_cat_id_reindex,'rb') as f:
    rec_cat_id_reindex=pickle.load(f)

In [18]:
#4.2padding
padded_rec_cat_id=Padding(rec_cat_id_reindex,max_len,lens)

In [19]:
#5定义模型
class RecPreferenceModel(nn.Module):
    def __init__(self,embed_size_user,embed_size_cat,hidden_size,
                size_user,size_cat,size_poi,
                num_layers,dropout=0.1):
        super(RecPreferenceModel,self).__init__()
        
        self.embed_user=nn.Embedding(size_user,embed_size_user)
        self.embed_cat=nn.Embedding(size_cat,embed_size_cat)
        
        lstm_size=embed_size_user+embed_size_cat
        self.lstm=nn.LSTM(lstm_size,hidden_size,num_layers,batch_first=True,dropout=dropout)
        self.fc=nn.Linear(hidden_size,size_poi)
    
    def forward(self,inputs_user,inputs_cat,inputs_delta_t,inputs_delta_d):
        '''
        inputs_cat、inputs_delta_t,inputs_delta_d是padding之后的
        '''
        inputs_user=self.embed_user(inputs_user).unsqueeze(1).repeat(1,inputs_cat.size(1),1)#(n_user,1)->(n_user,seq_len,1)
        inputs_cat=self.embed_cat(inputs_cat)#(n_user,seq_len,embed_cat)
        inputs_delta_t=inputs_delta_t.unsqueeze(2)#(n_users,seq_len,1)
        inputs_delta_d=inputs_delta_d.unsqueeze(2)#(n_users,seq_len,1)
        inputs=torch.cat((inputs_user,inputs_cat,inputs_delta_t,inputs_delta_d),2)
        output,_=self.lstm(inputs)
        out=self.fc(output)
        return out

In [20]:
#5.1验证模型
##5.1.1声明模型
hidden_size=128
num_layers=1
size_poi=3906
rec_pre_model=RecPreferenceModel(embed_size_user,embed_size_cat,hidden_size,size_user,size_cat,size_poi,num_layers)

/home/jovyan/.conda/envs/d2l/lib/python3.9/site-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [21]:
##5.1.2处理待传入数据
###5.1.2.1users，可以直接传
###5.1.2.2cat_id在传入前需要用重编码之后的所有数据，padding
#检查padded_reindex_cat_id数据
print(padded_rec_cat_id)

tensor([[ 24,   5,   5,  ...,  11,  29,  21],
        [ 48,   5,  31,  ...,  53,  54,  33],
        [ 10,  24,   2,  ...,  67,  61,  60],
        ...,
        [ 30,  70,  30,  ...,  40,   0,   0],
        [ 47,  47, 125,  ...,  33,  33,  83],
        [ 10,  10,  95,  ...,  16,  10,  69]], dtype=torch.int32)


In [22]:
#5.1.2.3传入padded_rec_delta_t
print(padded_rec_delta_t)

tensor([[   67613,    87274,   356947,  ...,   435024, 10751110,   162465],
        [ 7512611,  3876121,    24153,  ...,   685830,    13488,     3423],
        [    4092,   344985,   253180,  ...,    75781,   476178,  3580557],
        ...,
        [  444307,   665732,   333925,  ...,   485888,        0,        0],
        [     465,       47,   129815,  ...,      140,   206693,   929301],
        [  102170,    17419,  1082673,  ...,     1742,   499759,    49031]],
       dtype=torch.int32)


In [23]:
out=rec_pre_model(users,padded_rec_cat_id,padded_rec_delta_t,padded_rec_delta_d)

In [24]:
print(size_cat)

285
